# Aufgabe 7: AutoML

Unter AutoML versteht man...

https://www.ml4aad.org/automl/

###### (a) Wählen Sie ein AutoML Package. Begründen Sie Ihre Auswahl.

Wir haben uns für _H2O AutoML_ entschieden, weil es in Hinblick auf die Reproduzierbarkeit der Ergebnisse am leichtesten auf verschiedenen Plattformen installierbar ist.

Bei _auto-sklearn_ beispielsweise fehlten die Berechtigungen, um die benötigten Dependencies auf SDIL zu installieren.
Für andere AutoML-Bibliotheken wurde lediglich ein git-repo zur Verfügung gestellt, weshalb diese nicht einfach über eine `requirements.txt` installiert werden können.

Zudem fanden wir die Dokumentation zu _H2O AutoML_ auf den ersten Blick sehr übersichtlich.

###### (b) Führen Sie die Klassifikationsaufgabe von Aufgabe 3 mit AutoML durch. Vergleichen Sie die Ergebnisse mit den Ergebnissen aus Aufgabe 3.

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [1]:
# Import Libraries
import time
import multiprocessing

import pandas as pd
import numpy as np

from sklearn import svm
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.3" 2019-04-16; OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.10.1); OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.10.1, mixed mode, sharing)
  Starting server from /home/carmen/anaconda3/envs/python3_5-SmartDataAnalytics/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpu3mte_hz
  JVM stdout: /tmp/tmpu3mte_hz/h2o_carmen_started_from_python.out
  JVM stderr: /tmp/tmpu3mte_hz/h2o_carmen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,4 months and 2 days !!!
H2O cluster name:,H2O_from_python_carmen_9aus3f
H2O cluster total nodes:,1
H2O cluster free memory:,3.889 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
# Load dataset iris
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
iris = pd.read_csv(url, names=names)

In [63]:
# Split-out validation dataset
array = iris.values
X = array[:,0:4]
Y = array[:,4]
test_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
Y
iris

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [5]:
# prepare data by creating a train and test array and transforming it into an h2o frame
train_array = np.column_stack([X_train, y_train])
test_array = np.column_stack([X_test, y_test])

train = H2OFrame(python_obj=train_array, column_names=names)
test = H2OFrame(python_obj=test_array, column_names=names)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# Identify predictors and response
x = names
y = "class"
x.remove(y)

In [7]:
# set number of models to 20 and start training
aml = H2OAutoML(max_models=20, seed=1, nfolds=10)
aml.train(x=x, y=y, training_frame=train)                                 

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
display(lb.head(rows=lb.nrows)) # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20190521_075633_model_2,0.00877193,0.045072,0.110934,0.0123064
DeepLearning_grid_1_AutoML_20190521_075633_model_1,0.00877193,0.119951,0.106519,0.0113463
DRF_1_AutoML_20190521_075633,0.017094,0.0615018,0.135776,0.0184352
XRT_1_AutoML_20190521_075633,0.017094,0.0564404,0.130203,0.0169529
StackedEnsemble_BestOfFamily_AutoML_20190521_075633,0.0173189,0.145879,0.159386,0.0254038
DeepLearning_grid_1_AutoML_20190521_075633_model_6,0.0173189,0.0611777,0.127457,0.0162452
StackedEnsemble_AllModels_AutoML_20190521_075633,0.0173189,0.128639,0.142999,0.0204487
DeepLearning_grid_1_AutoML_20190521_075633_model_5,0.0175439,0.156639,0.126343,0.0159625
DeepLearning_grid_1_AutoML_20190521_075633_model_4,0.0175439,0.0569369,0.132442,0.0175409
DeepLearning_grid_1_AutoML_20190521_075633_model_3,0.0175439,0.395078,0.13946,0.019449


In [9]:
# The leader model is stored here
display(aml.leader)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_20190521_075633_model_2


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.003373211996851959
RMSE: 0.058079359473499355
LogLoss: 0.014008059018088633
Mean Per-Class Error: 0.0
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
43.0,0.0,0.0,0.0,0 / 43
0.0,38.0,0.0,0.0,0 / 38
0.0,0.0,39.0,0.0,0 / 39
43.0,38.0,39.0,0.0,0 / 120


Top-3 Hit Ratios: 


k,hit_ratio
1,1.0
2,1.0
3,1.0



ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.0123063811231901
RMSE: 0.11093412965895617
LogLoss: 0.04507203076061261
Mean Per-Class Error: 0.008771929824561403
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
accuracy,0.9916667,0.0176777,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.9166667,1.0
err,0.0083333,0.0176777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0833333,0.0
err_count,0.1,0.2121320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
logloss,0.0450720,0.0539093,0.0000543,0.0030583,0.0003478,0.0070887,0.0785683,0.0004371,0.0580536,0.0384664,0.2594796,0.0051661
max_per_class_error,0.0166667,0.0353553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1666667,0.0
mean_per_class_accuracy,0.9944444,0.0117851,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.9444444,1.0
mean_per_class_error,0.0055556,0.0117851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0555556,0.0
mse,0.0123064,0.0168814,0.0000000,0.0000776,0.0000008,0.0001867,0.0168832,0.0000009,0.0140163,0.0101798,0.0814176,0.0003008
r2,0.9768405,0.0337371,1.0,0.9998654,0.9999983,0.9997285,0.9707086,0.9999988,0.9789755,0.9823386,0.8371648,0.9996266
rmse,0.0676360,0.0621762,0.0001330,0.0088085,0.0009052,0.0136625,0.1299355,0.0009479,0.1183907,0.1008951,0.2853377,0.0173443


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
,2019-05-21 08:08:40,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-05-21 08:08:40,3 min 49.965 sec,44444 obs/sec,10.0,1,1200.0,0.2217859,0.1526704,0.9278989,0.075
,2019-05-21 08:08:45,3 min 54.983 sec,44991 obs/sec,1890.0,189,226800.0,0.1573619,0.0829419,0.9637028,0.0333333
,2019-05-21 08:08:50,3 min 59.997 sec,45832 obs/sec,3840.0,384,460800.0,0.1125215,0.0490004,0.9814414,0.0083333
,2019-05-21 08:08:55,4 min 4.998 sec,45755 obs/sec,5740.0,574,688800.0,0.0814069,0.0267805,0.9902860,0.0083333
,2019-05-21 08:09:00,4 min 10.013 sec,45864 obs/sec,7670.0,767,920400.0,0.0733713,0.0198780,0.9921091,0.0083333
,2019-05-21 08:09:02,4 min 12.480 sec,46114 obs/sec,8660.0,866,1039200.0,0.0580794,0.0140081,0.9950556,0.0


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sepal-length,1.0,1.0,0.2673093
sepal-width,0.9596403,0.9596403,0.2565208
petal-width,0.9045843,0.9045843,0.2418038
petal-length,0.8767595,0.8767595,0.2343660


In [10]:
# predict on test data
predictions = aml.leader.predict(test).as_data_frame()['predict'].tolist()

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [11]:
# get target for test data
y_test = test.as_data_frame()['class'].tolist()

In [12]:
# print results
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.8666666666666667
[[ 6  1  0]
 [ 0 10  2]
 [ 0  1 10]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      0.86      0.92         7
Iris-versicolor       0.83      0.83      0.83        12
 Iris-virginica       0.83      0.91      0.87        11

       accuracy                           0.87        30
      macro avg       0.89      0.87      0.88        30
   weighted avg       0.87      0.87      0.87        30



Der DeepLearning-Ansatz wird von AutoML als der geeignetste betrachtet. Dieser erreicht auf den Trainingsdaten eine Accuracy von 0.8667 und liegt damit unterhalb der Accuracy des Models mit der höchsten Accuracy aus Aufgabe drei (0.967). 

# Zweiter Datensatz

In [66]:
# Load dataset
url_heart = "./data/heart.csv"
names_heart =  ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 
                 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved', 'exercise_induced_angina',
                 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']
heart = pd.read_csv(url_heart, names=names_heart)
heart.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
1,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
2,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
3,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
4,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1


In [70]:
# drop first row
heart = heart.drop(heart.index[0])
heart.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
1,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
2,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
3,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
4,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
5,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [73]:
# Split-out validation dataset
array_heart = heart.values
X_heart = array_heart[:,0:13]
Y_heart = array_heart[:,13]
test_size_heart = 0.20
seed_heart = 7
X_heart_train, X_heart_test, y_heart_train, y_heart_test = model_selection.train_test_split(X_heart, Y_heart, test_size=test_size_heart, random_state=seed_heart)

In [76]:
# prepare data by creating a train and test array and transforming it into an h2o frame
train_heart_array = np.column_stack([X_heart_train, y_heart_train])
test_heart_array = np.column_stack([X_heart_test, y_heart_test])

train_heart = H2OFrame(python_obj=train_heart_array, column_names=names_heart)
test_heart = H2OFrame(python_obj=test_heart_array, column_names=names_heart)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [77]:
# Identify predictors and response
x_heart = names_heart
y_heart = "target"
x_heart.remove(y_heart)

In [78]:
# set number of models to 20 and start training
aml_heart = H2OAutoML(max_models=20, seed=1, nfolds=10)
aml_heart.train(x=x_heart, y=y_heart, training_frame=train_heart)                                 

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [79]:
# View the AutoML Leaderboard
lb_heart = aml_heart.leaderboard
display(lb_heart.head(rows=lb_heart.nrows)) # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190521_094939,0.0972425,0.311837,0.0972425,0.227896,0.220137
StackedEnsemble_BestOfFamily_AutoML_20190521_094939,0.0984062,0.313698,0.0984062,0.216939,0.219772
DeepLearning_grid_1_AutoML_20190521_094939_model_4,0.10458,0.323388,0.10458,0.215906,0.227562
DeepLearning_grid_1_AutoML_20190521_094939_model_5,0.105891,0.325409,0.105891,0.237705,0.226421
GBM_2_AutoML_20190521_094939,0.107009,0.327122,0.107009,0.22923,0.228049
DeepLearning_grid_1_AutoML_20190521_094939_model_2,0.108185,0.328915,0.108185,0.235225,0.241405
GBM_4_AutoML_20190521_094939,0.109222,0.330487,0.109222,0.232814,0.229984
GBM_3_AutoML_20190521_094939,0.109397,0.330751,0.109397,0.23277,0.230064
GBM_1_AutoML_20190521_094939,0.11113,0.333362,0.11113,0.221798,0.231919
DeepLearning_grid_1_AutoML_20190521_094939_model_3,0.117042,0.342114,0.117042,0.22099,0.241487


In [80]:
# The leader model is stored here
display(aml_heart.leader)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190521_094939
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.017411417595183817
RMSE: 0.1319523307682885
MAE: 0.12258900279846072
RMSLE: 0.08920483261200277
R^2: 0.9295410268073283
Mean Residual Deviance: 0.017411417595183817
Null degrees of freedom: 241
Residual degrees of freedom: 234
Null deviance: 59.801652892561854
Residual deviance: 4.213563058034484
AIC: -275.48599317552697

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.09724250544196439
RMSE: 0.31183730604590015
MAE: 0.2278955161979094
RMSLE: 0.22013659226855617
R^2: 0.606487694257656
Mean Residual Deviance: 0.09724250544196439
Null degrees of freedom: 241
Residual degrees of freedom: 233
Null deviance: 60.32145620630827
Residual deviance: 23.53268631695538
AIC: 142.7737879062629


In [81]:
# predict on test data
predictions_heart = aml_heart.leader.predict(test_heart).as_data_frame()['predict'].tolist()

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [114]:
# get target for test data
y_heart_test = test_heart.as_data_frame()['target'].tolist()

# predictions are continuous --> set threshold for final prediction to 0.5
predictions_heart = pd.DataFrame(predictions_heart)
mask = predictions_heart>0.5
predictions_heart_binary = mask.astype(int)

In [117]:
# print results
print(accuracy_score(y_heart_test, predictions_heart_binary))
print(confusion_matrix(y_heart_test, predictions_heart_binary))
print(classification_report(y_heart_test, predictions_heart_binary))

0.7377049180327869
[[18 12]
 [ 4 27]]
              precision    recall  f1-score   support

           0       0.82      0.60      0.69        30
           1       0.69      0.87      0.77        31

    accuracy                           0.74        61
   macro avg       0.76      0.74      0.73        61
weighted avg       0.75      0.74      0.73        61



Auch hier ist die Accuracy mit ca. 0.7377 um etwa 10 Prozentpunkte schlechter als in Aufgabe 3.

###### (c) Was ist Ihre Meinung zu AutoML?

- AutoML nimmt einem viel Arbeit ab, was gerade für Anfänger hilfreich ist. Allerdings verliert man hierdurch auch etwas Kontrolle. Die Ergebnisse sind scheinbar schlechter als in Aufgabe 3. Jedoch ist es schwer nachzuvollziehen wieso, da eben alles automatisch passiert. Zudem ist nicht klar, wieso welche Modelle getestet wurden. 

- Außerdem ist die Darstellung des aml.leader() seltsam. Leider wurde dafür kein Fix gefunden 

- Das Training dauert relativ lange, weshalb es etwas enttäuschend ist, die Ergebnisse aus Aufgabe 3 nicht toppen zu können. 


- "goal of finding the “best” model without any prior knowledge", https://medium.com/analytics-vidhya/gentle-introduction-to-automl-from-h2o-ai-a42b393b4ba2 --> diese Erfahrung können wir nicht teilen. Man kann mit wenig Ahnung ein akzeptables Ergebnis erreichen, jedoch waren unsere Ergebnisse nicht im sehr guten Bereich.

-"The current version of AutoML trains and cross-validates the following algorithms (in the following order): three pre-specified XGBoost GBM (Gradient Boosting Machine) models, a fixed grid of GLMs, a default Random Forest (DRF), five pre-specified H2O GBMs, a near-default Deep Neural Net, an Extremely Randomized Forest (XRT), a random grid of XGBoost GBMs, a random grid of H2O GBMs, and a random grid of Deep Neural Nets. In some cases, there will not be enough time to complete all the algorithms, so some may be missing from teh leaderboard. AutoML then trains two Stacked Ensemble models." http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html --> Unseres Erachtens nach sollten noch mehr verschiedene Modelle ausprobiert werden. 